In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import git

In [2]:
# variables and files to be apply horizontal averaging to
hor_avg_metadata_list = [
    {'varnames':['thetao'],
     'fname_in':'../omip1/thetao_Oyr_CESM2_omip1_r2i1p1f1_gr_0311-0372.nc',
     'fname_out':'../omip1/thetao_Oyr_CESM2_omip1_r2i1p1f1_gr_700m_0311-0372.nc'},
    {'varnames':['thetao'],
     'fname_in':'../omip2/thetao_Oyr_CESM2_omip2_r1i1p1f1_gr_0306-0366.nc',
     'fname_out':'../omip2/thetao_Oyr_CESM2_omip2_r1i1p1f1_gr_700m_0306-0366.nc'},
]

In [3]:
for hor_avg_metadata in hor_avg_metadata_list:
    ds_in = xr.open_dataset(hor_avg_metadata['fname_in'], chunks={'time':1})
    ds_out = xr.Dataset()
    lev_weight = np.minimum(ds_in.lev_bnds.sel(lev=slice(0,700)), 700.0).diff(dim='d2').squeeze(dim='d2')
    for varname in hor_avg_metadata['varnames']:
        da_in = ds_in[varname].sel(lev=slice(0,700))
        da_out = (da_in * lev_weight).sum(dim='lev')
        ones_masked = xr.ones_like(da_in).where(da_in.notnull())
        denom = (ones_masked * lev_weight).sum(dim='lev')
        da_out /= denom
        da_out.attrs = da_in.attrs

        # delete some attributes that do not make sense for depth means
        for attr_name in ['cell_measures', 'cell_methods', 'comment']:
            del da_out.attrs[attr_name]

        # propagate some encoding info from da_in
        for key in ['dtype', '_FillValue', 'zlib', 'complevel']:
            if key in da_in.encoding:
                da_out.encoding[key] = da_in.encoding[key]
        if 'missing_value' in da_in.encoding:
            da_out.attrs['missing_value'] = np.float32(da_in.encoding['missing_value'])

        ds_out[varname] = da_out

    # if any variable in ds_out has a bounds attribute, copy the bounds variable over
    for varname in ds_out.variables:
        if 'bounds' in ds_out[varname].attrs:
            bounds_name = ds_out[varname].attrs['bounds']
            ds_out[bounds_name] = ds_in[bounds_name]

    # if a variable doesn't have _FillValue in the encoding, set _FillValue to None
    for varname in ds_out.variables:
        if '_FillValue' not in ds_out[varname].encoding:
            ds_out[varname].encoding['_FillValue'] = None

    ds_out.attrs = ds_in.attrs
    ds_out.attrs['contact'] = 'klindsay@ucar.edu'
    ds_out.attrs['githash'] = git.Repo(search_parent_directories=True).head.object.hexsha

    ds_out.to_netcdf(hor_avg_metadata['fname_out'], unlimited_dims='time')

/glade/work/klindsay/miniconda3/envs/analysis/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
/glade/work/klindsay/miniconda3/envs/analysis/lib/python3.7/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in true_divide
  return func(*args2)
